In [1]:
import torch
import torchvision
import sys
if '/home/dennis/git_repos/mslp-dataset/benchmark/nn' in sys.path:
    sys.path.remove('/home/dennis/git_repos/mslp-dataset/benchmark/nn')
sys.path.insert(0, '/home/dennis/git_repos/mslp-dataset/benchmark/nn')
from utils.dataset import MARITIMEDETECTION
from models.faster_rcnn import LitFasterRCNN
from utils.transforms import ToTensor
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from pprint import pprint
from tqdm import tqdm

### Evaluate Faster RCNN results with Mean Average Precision
* best result with lr: 0.0001

In [2]:
test_dataset = MARITIMEDETECTION("/media/dennis/3B9FC6C559F7A944/converted_object_detection_dataset", "test",
                                    transform=torchvision.transforms.Compose([ 
                                    ToTensor()]))

model = LitFasterRCNN.load_from_checkpoint("/home/dennis/git_repos/mslp-dataset/benchmark/nn/faster_rcnn_log/version_0/fasterrcnn-epoch=38-val_loss=0.00.ckpt", map_location="cuda")
model.eval()
metric = MeanAveragePrecision(class_metrics=True)

100%|██████████| 310/310 [00:01<00:00, 248.19it/s]
/media/dennis/3B9FC6C559F7A944/anaconda3/envs/benchmark/lib/python3.9/site-packages/pytorch_lightning/utilities/migration/utils.py:52: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.7, which is newer than your current Lightning version: v2.0.6
  rank_zero_warn(


In [3]:
model = model.cuda()
with torch.no_grad():
    for sample in tqdm(test_dataset):
        img, targets = sample["left_img"], sample["targets"]

        output = model(img.cuda(), None)

        scores = output[0]["scores"].cpu()
        boxes = output[0]["boxes"].cpu()
        labels = output[0]["labels"].cpu()

        score_mask = scores > 0.8
        scores = scores[score_mask]
        boxes = boxes[score_mask]
        labels_int = labels[score_mask]
        labels = [str(l) for l in labels_int.tolist()]

        image = sample["left_img"]
        image = (image*255).to(torch.uint8)

        gt_mask = targets[0].sum(dim=1) != 0
        gt_boxes = targets[0][gt_mask]
        gt_labels_int = targets[1][gt_mask].to(torch.int32).tolist()
        gt_labels = [str(num) for num in gt_labels_int]
        
        preds = [dict(boxes=boxes, scores=scores, labels=labels_int)]
        target = [dict(boxes=gt_boxes, labels=torch.tensor(gt_labels_int))]
        metric.update(preds, target)

pprint(metric.compute())


100%|██████████| 310/310 [01:10<00:00,  4.40it/s]


{'classes': tensor([1, 2, 3, 4, 5, 6, 8, 9], dtype=torch.int32),
 'map': tensor(0.2560),
 'map_50': tensor(0.5014),
 'map_75': tensor(0.2207),
 'map_large': tensor(0.2650),
 'map_medium': tensor(0.0953),
 'map_per_class': tensor([0.1471, 0.4578, 0.2759, 0.0000, 0.3558, 0.2777, 0.2807, 0.2528]),
 'map_small': tensor(0.),
 'mar_1': tensor(0.3121),
 'mar_10': tensor(0.3241),
 'mar_100': tensor(0.3241),
 'mar_100_per_class': tensor([0.2221, 0.5708, 0.3759, 0.0000, 0.3659, 0.3390, 0.4241, 0.2953]),
 'mar_large': tensor(0.3139),
 'mar_medium': tensor(0.2358),
 'mar_small': tensor(0.)}
